<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202502/blob/master/personalidad/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tema 01: Carga de datos

### Importando base de datos

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
# API client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url = 'https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing'
gsheets = gc.open_by_url(url)
sheets = gsheets.worksheet('Respuestas de formulario 1').get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo a data frame
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

### Inspección del data frame

In [ ]:
type(df)

In [ ]:
df.head()

In [ ]:
# Ver la estructura del df
df.info()

In [ ]:
# Tupla con la cantidad de filas y columnas
df.shape

In [ ]:
# Cantidad de filas
len(df)

In [ ]:
# Cantidad de columnas
len(df.columns)

In [ ]:
df.columns

## Tema 02: Transformación de datos

### Valores perdidos

Identificando NAs

In [ ]:
df[['Escribe tu edad exacta']]

In [ ]:
# Los NAs están como strings vacíos
import numpy as np
df.replace('', np.nan, inplace=True)

In [ ]:
df.info()

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

Tratando los NAs

In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors='coerce')

In [ ]:
df['Escribe tu edad exacta'].dtype

Imputacion por el promedio

In [ ]:
# Calcular la media
edad_promedio = df['Escribe tu edad exacta'].mean().round(0)
edad_promedio

In [ ]:
# Creando un nuevo df
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2['edad2'] = df2['Escribe tu edad exacta'].fillna(edad_promedio)

In [ ]:
df2[['Escribe tu edad exacta', 'edad2']]

In [ ]:
df2.info()

In [ ]:
# Relocalizar la columna 'edad2' después de 'Escribe tu edad exacta'
# lista_columnas = list(df2.columns).index('Escribe tu edad exacta') + 1
# lista_columnas

In [ ]:
# df2.insert(lista_columnas, 'edad2', df2.pop('edad2'))

In [ ]:
# df2.info()

Creando relocate en py

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns).index(after) + 1
  df.insert(lista_columnas, columna, df.pop(columna))
  return df

In [ ]:
relocate(
    df = df2,
    columna = 'edad2',
    after = 'Escribe tu edad exacta'
)

In [ ]:
df2.columns

Eliminando las filas que contienen NAs

In [ ]:
df2.shape

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.shape

### Estandarización de variables

#### Normalización

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
df2[['edad2']]

In [ ]:
# normalizando
normalizador.fit_transform(df2[['edad2']])

In [ ]:
# creando df3
df3 = df2.copy()

In [ ]:
df3['edadZ'] = normalizador.fit_transform(df3[['edad2']])

In [ ]:
df3 = relocate(
    df = df3,
    columna = 'edadZ',
    after = 'edad2'
)

In [ ]:
df3.info()

#### Rango

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Instanciando MinMaxScaler()
rango = MinMaxScaler()

In [52]:
df3['edad_rango'] = rango.fit_transform(df3[['edad2']])

In [53]:
df3 = relocate(
    df = df3,
    columna = 'edad_rango',
    after = 'edadZ'
)

In [ ]:
df3.info()

In [ ]:
df3[['edad2', 'edadZ', 'edad_rango']].head()

### Agrupaciones

#### Numéricas

In [ ]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ['18 o menos', '19 a 21', 'Más de 21']

In [ ]:
df3['edadGR'] = pd.cut(df3['edad2'], bins=cortes, labels=etiquetas)

In [ ]:
df3.value_counts('edadGR', normalize=True)

In [ ]:
df3[['edad2', 'edadZ', 'edad_rango', 'edadGR']].head()

In [91]:
df3 = relocate(
    df = df3,
    columna = 'edadGR',
    after = 'edad_rango'
)

In [ ]:
df3.info()

#### Categóricas

In [ ]:
df3.info()

In [ ]:
pd.unique(df3['Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar]'])

In [ ]:
pd.unique(df3.iloc[:,12])

In [ ]:
df3['Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme]'].value_counts()

In [ ]:
df3.iloc[:,15].value_counts()

Top 2 Box

In [ ]:
df3.iloc[:,8].value_counts()

In [ ]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero']).sum()

In [77]:
# Funcion condicional para top2box
def top2box(x):
  if x in ['Totalmente verdadero', 'Un poco verdadero']:
    return 1
  else:
    return 0

In [ ]:
# Aplicando la funcion
(
    df3.iloc[:,8]
    .apply(top2box)
    .value_counts()
)

In [ ]:
# Función lambda
df3.iloc[:,8].apply(
    lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0
).value_counts()

### De ordinal a numérica

In [83]:
import numpy as np

In [84]:
df3.iloc[:,8].unique()

array(['No lo sé', 'Totalmente verdadero', 'Un poco falso',
       'Un poco verdadero', 'Totalmente falso'], dtype=object)

In [85]:
# Condiciones
condiciones = [
    df3.iloc[:,8] == 'Totalmente verdadero',
    df3.iloc[:,8] == 'Un poco verdadero',
    df3.iloc[:,8] == 'No lo sé',
    df3.iloc[:,8] == 'Un poco falso',
    df3.iloc[:,8] == 'Totalmente falso'
]

In [87]:
# resultados
resultados = [5,4,3,2,1]

In [ ]:
# Usando select
numeric = np.select(condiciones, resultados, default=np.nan)
numeric

In [ ]:
numeric.mean()

In [ ]:
np.unique(numeric, return_counts=True)

## Bucles

### Forma standard

In [116]:
# Creando el df4
df4 = df3.copy()

In [ ]:
df4.info()

In [118]:
# Creando una lista vacía
frases = []

# bucle para llenar la lista
for col in df4.columns:
  if col.startswith('Según'):
    frases.append(col)

In [ ]:
frases

In [120]:
# aplicando la función top2box a las columnas que pertenecen a 'frase'
for frase in frases:
  df4[frase] = df4[frase].apply(top2box)

In [ ]:
df4[frases].head()

### Pythonic way | Forma pythonica

In [122]:
# Creando df5
df5 = df3.copy()

In [125]:
# Lista vacía
frases = []

# Comprehension list
frases = [col for col in df5.columns if col.startswith('Según')]

In [ ]:
frases

In [127]:
# Aplicando el bucle, esta vez con lambda
for frase in frases:
  df5[frase] = df5[frase].apply(lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0)

In [129]:
df5[frases] == df4[frases]

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Creo que la vida se trata de tomar riesgos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [En mi casa, la familia es muy importante, por eso paso mucho tiempo con ellos]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Tener dinero es clave para ser respetado],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comer sano y estar en forma],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Vestirse bien es clave para el éxito ],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy sincero con las personas, aunque la verdad les duela]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupo y ayudo a los necesitados],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Invierto mucho tiempo y dinero para aprender cosas nuevas],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas]
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
253,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
254,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
255,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


## Tema 03: Manipulación de datos

### Selección de columnas

In [ ]:
df5['Sexo']

In [ ]:
df5[['Sexo']]

In [ ]:
df5[['Sexo', 'edad2']]

In [ ]:
# eliminar columna
df5.drop(columns=['Marca temporal'])

In [ ]:
# Seleccionar cols que comienzan con... ¿Cuánto
df5[[col for col in df5.columns if col.startswith('¿Cuánto')]]

In [ ]:
# Seleccionar cols que terminan con un corchete
df5[[col for col in df5.columns if col.endswith(']')]]

In [ ]:
# contains
df5.filter(like='edad')

### Selección de filas